In [1]:
import torch
#torch.set_default_dtype(torch.float64)
import numpy as np
import ase.io
import tqdm

from pytorch_prototype.code_pytorch import *
from pytorch_prototype.utilities import *
from pytorch_prototype.clebsch_gordan import ClebschGordan
from pytorch_prototype.clebsch_combining import ClebschCombining

from sklearn.linear_model import Ridge
from matplotlib import pyplot as plt
from torch import vmap

In [2]:
LAMBDA_MAX = 2

HYPERS = {
    'interaction_cutoff': 6.3,
    'max_radial': 2,
    'max_angular': LAMBDA_MAX,
    'gaussian_sigma_type': 'Constant',
    'gaussian_sigma_constant': 0.05,
    'cutoff_smooth_width': 0.3,
    'radial_basis': 'GTO'
    
}
subset = '0:10'
DEVICE = 'cuda'
METHANE_PATH = '../methane.extxyz'
clebsch = ClebschGordan(LAMBDA_MAX)

In [3]:
structures = ase.io.read(METHANE_PATH , index=subset)
all_species = get_all_species(structures)
coefficients = get_coefs(structures, HYPERS, all_species)


L2_mean = get_L2_mean(coefficients)

for key in coefficients.keys():
    coefficients[key] /= torch.sqrt(L2_mean)
    coefficients[key] = coefficients[key].to(DEVICE).requires_grad_()
    
structural_indices = get_structural_indices(structures)


coef_der, central_indices, derivative_indices= \
get_coef_ders(structures, HYPERS, all_species)
for key in coef_der.keys():
    coef_der[key] /= torch.sqrt(L2_mean)
    coef_der[key] = coef_der[key].to(DEVICE)
    

In [4]:
class ModelVariousOutputs(torch.nn.Module):
    def __init__(self, clebsch):
        super(ModelVariousOutputs, self).__init__()
        self.clebsch_combining_first = ClebschCombining(clebsch, LAMBDA_MAX)
        self.clebsch_combining_second = ClebschCombining(clebsch, 0)
        self.linear = torch.nn.Linear(960, 1, bias = False)
        self.second_linear = torch.nn.Linear(48, 7 * 19 * 42)
        
    def forward(self, coefs):
        #X = L2_normalize(X)
        ps = self.clebsch_combining_first(coefs, coefs)
        ps_invariants = ps['0'].squeeze()
        bs = self.clebsch_combining_second(ps, coefs)['0'].squeeze()
        some = self.second_linear(ps_invariants)
        some = some.reshape([-1, 7, 19, 42])
        return {'vector output ps' : ps_invariants, 'vector output bs' : bs,
                'scalar output energies' : self.linear(bs).squeeze(),
                'multidimensional output' : some}


# structural predictions

In [5]:
block = ModelVariousOutputs(clebsch.precomputed_).to(DEVICE)
model = Atomistic(block).to(DEVICE)

predictions = model(coefficients, structural_indices = structural_indices, n_structures = 10)
print("predictions: ")
for key in predictions.keys():
    print(key, predictions[key].shape)
jacobians = model.get_jacobians(coef_der, central_indices, derivative_indices, 
                                         coefficients, structural_indices = structural_indices,
                                n_structures = 10)
print("jacobians")
for key in jacobians.keys():
    print(key, jacobians[key].shape)

predictions: 
vector output ps torch.Size([10, 48])
vector output bs torch.Size([10, 960])
scalar output energies torch.Size([10])
multidimensional output torch.Size([10, 7, 19, 42])


/home/pozdn/pytorch_prototype/pytorch_prototype/code_pytorch.py:107: UserWarning: torch.vmap is an experimental prototype that is subject to change and/or deletion. Please use at your own risk. There may be unexpected performance cliffs due to certain operators not being implemented. To see detailed performance warnings please use `torch._C._debug_only_display_vmap_fallback_warnings(True) before the call to `vmap`.
  result = list(vmap(vjp)(output_grads))


jacobians
vector output ps torch.Size([50, 3, 48])
vector output bs torch.Size([50, 3, 960])
scalar output energies torch.Size([50, 3])
multidimensional output torch.Size([50, 3, 7, 19, 42])


# atomic predictions

In [6]:
block = ModelVariousOutputs(clebsch.precomputed_).to(DEVICE)
model = Atomistic(block, accumulate = False).to(DEVICE)

predictions = model(coefficients, structural_indices = structural_indices)
print("predictions: ")
for key in predictions.keys():
    print(key, predictions[key].shape)
jacobians = model.get_jacobians(coef_der, central_indices, derivative_indices, 
                                         coefficients, structural_indices = structural_indices)
print("jacobians")
for key in jacobians.keys():
    print(key, jacobians[key].shape)

predictions: 
vector output ps torch.Size([50, 48])
vector output bs torch.Size([50, 960])
scalar output energies torch.Size([50])
multidimensional output torch.Size([50, 7, 19, 42])
jacobians
vector output ps torch.Size([50, 3, 48])
vector output bs torch.Size([50, 3, 960])
scalar output energies torch.Size([50, 3])
multidimensional output torch.Size([50, 3, 7, 19, 42])


# loop algorithm

In [7]:
block = ModelVariousOutputs(clebsch.precomputed_).to(DEVICE)
model = Atomistic(block).to(DEVICE)

predictions = model(coefficients, structural_indices = structural_indices, n_structures = 10)
print("predictions: ")
for key in predictions.keys():
    print(key, predictions[key].shape)
jacobians = model.get_jacobians(coef_der, central_indices, derivative_indices, 
                                         coefficients, structural_indices = structural_indices,
                                n_structures = 10, algorithm = 'loops')
print("jacobians")
for key in jacobians.keys():
    print(key, jacobians[key].shape)

predictions: 
vector output ps torch.Size([10, 48])
vector output bs torch.Size([10, 960])
scalar output energies torch.Size([10])
multidimensional output torch.Size([10, 7, 19, 42])


100%|██████████| 5586/5586 [00:03<00:00, 1788.26it/s]

jacobians
vector output ps torch.Size([50, 3, 48])
vector output bs torch.Size([50, 3, 960])
scalar output energies torch.Size([50, 3])
multidimensional output torch.Size([50, 3, 7, 19, 42])


In [8]:
block = ModelVariousOutputs(clebsch.precomputed_).to(DEVICE)
model = Atomistic(block, accumulate = False).to(DEVICE)

predictions = model(coefficients, structural_indices = structural_indices)
print("predictions: ")
for key in predictions.keys():
    print(key, predictions[key].shape)
jacobians = model.get_jacobians(coef_der, central_indices, derivative_indices, 
                                         coefficients, structural_indices = structural_indices,
                                algorithm = 'loops')
print("jacobians")
for key in jacobians.keys():
    print(key, jacobians[key].shape)

predictions: 
vector output ps torch.Size([50, 48])
vector output bs torch.Size([50, 960])
scalar output energies torch.Size([50])
multidimensional output torch.Size([50, 7, 19, 42])


100%|██████████| 5586/5586 [00:02<00:00, 1921.51it/s]


jacobians
vector output ps torch.Size([50, 3, 48])
vector output bs torch.Size([50, 3, 960])
scalar output energies torch.Size([50, 3])
multidimensional output torch.Size([50, 3, 7, 19, 42])
